<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/machine_learning_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 線性迴歸

In [1]:
# 下載資料集並解壓縮
!gdown '1xqOapGi_kPIue0C5r4zDS1fe57tWo-2P' --output ML.zip
!unzip ML.zip

Downloading...
From (original): https://drive.google.com/uc?id=1xqOapGi_kPIue0C5r4zDS1fe57tWo-2P
From (redirected): https://drive.google.com/uc?id=1xqOapGi_kPIue0C5r4zDS1fe57tWo-2P&confirm=t&uuid=6f244e38-6b3d-4de4-8673-c719c8e3c99f
To: /content/ML.zip
100% 44.6M/44.6M [00:00<00:00, 121MB/s]
Archive:  ML.zip
  inflating: housePrice.csv          
  inflating: mnist500.zip            
  inflating: mnist500_png.zip        
  inflating: toutiao_cat_data.txt    
  inflating: иTиоиолм╕ъо╞└╔.csv  
  inflating: л╚дс┴p╡╕кмкp╕ъо╞└╔.csv  
  inflating: breastCancer.csv        
  inflating: customer.csv            


In [2]:
# 更改亂碼檔名
!mv иTиоиолм╕ъо╞└╔.csv 汽車車型資料檔.csv
!mv л╚дс┴p╡╕кмкp╕ъо╞└╔.csv 客戶聯絡狀況資料檔.csv

## 波士頓房價預測

In [3]:
import pandas as pd
hp = pd.read_csv('/content/housePrice.csv')
hp

,犯罪率,豪宅比,公設比,臨公園,NO濃度,房間數,屋齡,賣場距離,捷運距離,繳稅率,師生比,低收入比,房價
0,2.64981,0.0,28.18,0,0.589,5.867,7.4,12.5002,5,102,21.0,23.038918,23.1
1,0.14511,0.0,6.95,0,0.213,4.283,81.3,14.6909,23,158,22.8,39.632374,15.8
2,3.93046,6.0,46.29,1,0.278,4.287,16.4,0.7680,21,637,7.0,37.513949,22.1
3,2.95667,0.0,15.28,0,0.355,1.154,76.4,9.1349,8,337,18.1,12.331483,16.6
4,1.24919,3.0,39.17,0,0.206,6.788,10.0,10.5811,17,222,17.9,12.984651,26.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.64935,0.0,13.13,0,0.420,4.671,29.6,2.7699,22,831,17.1,16.196702,23.0
496,3.81123,70.0,29.51,0,0.398,5.472,78.3,6.1768,9,928,19.5,8.527868,23.0
497,0.72696,0.0,56.88,0,0.628,3.126,91.4,13.3371,13,201,21.6,22.979185,18.1
498,9.52366,0.0,23.17,0,0.892,2.356,79.4,8.5226,23,907,18.9,6.263589,13.0


In [5]:
x = hp.drop('房價', axis=1)
y = hp['房價']

線性迴歸應進行標準化，因為不同特徵的數值相差很大

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# 資料先進行標準化
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
std_x = StandardScaler()
x_train = std_x.fit_transform(x_train) # 注意：訓練集要用 fit_transform
x_test = std_x.transform(x_test) # 測試集則用 transform
std_y = StandardScaler()
y_train = std_y.fit_transform(y_train.to_numpy().reshape(-1, 1)) # 分類時，目標值不需標準化，但迴歸時因為目標值是數值，所以也需標準化
y_test = std_y.transform(y_test.to_numpy().reshape(-1, 1))

這個問題問得非常好！這個指令背後其實牽涉到 **資料形狀（shape）** 和 **`scikit-learn` 的需求**，我們來一點一點解釋：

---

## 🔧 指令回顧：

```python
y_train = std_y.fit_transform(y_train.to_numpy().reshape(-1, 1))
```

---

### 🧠 為什麼要加上 `.to_numpy()`？

這通常表示 `y_train` 是一個 **Pandas Series**，但 `scikit-learn` 的 `fit_transform()`（像 `StandardScaler`）**期望輸入的是 2D 陣列（如 NumPy array）**，所以你要先轉換成 NumPy：

```python
y_train.to_numpy()
```

這會變成 shape 是 `(n_samples,)` 的一維陣列。

---

### 🧠 為什麼要 `reshape(-1, 1)`？

這是最關鍵的一步！  
`scikit-learn` 的大部分轉換器（像 `StandardScaler`）要求輸入 shape 是 `(n_samples, n_features)` 的 **二維陣列**。

但 `y_train` 是一個目標變數（target），通常是一維的。  
例如：

```python
y_train = [10, 12, 15, 20]
y_train.shape  # (4,)
```

這會讓 `fit_transform()` 拋錯，因為它期望的是像這樣：

```python
[[10],
 [12],
 [15],
 [20]]  # shape = (4, 1)
```

所以加上 `.reshape(-1, 1)`，就可以轉換成 `(4, 1)` 的二維格式，讓標準化處理順利執行。

---

### ✅ 總結：

| 步驟 | 目的 |
|------|------|
| `.to_numpy()` | 把 Pandas Series 轉成 NumPy array |
| `.reshape(-1, 1)` | 讓資料從一維 `(n,)` 變成二維 `(n, 1)`，符合 `scikit-learn` 的要求 |
| `fit_transform()` | 使用 `StandardScaler` 或其他轉換器來標準化數值 |

---

### 🎯 額外補充小技巧：

如果你在處理的是 X（特徵矩陣），通常本來就是 2D，不用 reshape；  
但如果是 y（目標變數），就幾乎一定要 reshape 才能通過轉換器。




In [7]:
sgd = SGDRegressor()
sgd.fit(x_train, y_train)
y_predict = std_y.inverse_transform(sgd.predict(x_test).reshape(-1, 1)) # 要利用 inverse_transform 將標準化後的值還原，才能知道原始的房價數字
y_real = std_y.inverse_transform(y_test)
for i in range(10):
  print('預測值：{}，真實值：{}'.format(y_predict[i], y_real[i]))

預測值：[23.3865153]，真實值：[25.1]
預測值：[16.12440629]，真實值：[16.8]
預測值：[9.95258436]，真實值：[14.4]
預測值：[23.49929288]，真實值：[23.1]
預測值：[21.22526918]，真實值：[17.5]
預測值：[18.03309386]，真實值：[17.7]
預測值：[26.50323173]，真實值：[22.2]
預測值：[22.63504481]，真實值：[25.7]
預測值：[21.64713248]，真實值：[16.5]
預測值：[18.94881727]，真實值：[18.9]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# 權重值
sgd.coef_

array([-0.22003353,  0.26942011, -0.04391603,  0.1300362 , -0.22736006,
        0.24173555, -0.28155999, -0.24484905, -0.25159509, -0.04135897,
       -0.00420241, -0.04087311])

In [ ]:
# 偏置值
sgd.intercept_

改善方法：

因為房價線性的機率很低